This notebook provides examples to go along with the [textbook](https://underactuated.csail.mit.edu/trajopt.html).  I recommend having both windows open, side-by-side!

# Trajectory optimization for the double integrator

In [ ]:
import sys
import time

import matplotlib.pyplot as plt
import numpy as np
import pydot
from IPython.display import HTML, SVG, clear_output, display
from pydrake.all import (
    Binding,
    Box,
    DiagramBuilder,
    DirectCollocation,
    DirectTranscription,
    FiniteHorizonLinearQuadraticRegulatorOptions,
    GraphOfConvexSets,
    GraphOfConvexSetsOptions,
    GurobiSolver,
    HPolyhedron,
    L2NormCost,
    LinearSystem,
    LogVectorOutput,
    MakeFiniteHorizonLinearQuadraticRegulator,
    MakeFirstAvailableSolver,
    MathematicalProgram,
    MosekSolver,
    MultibodyPlant,
    MultibodyPositionToGeometryPose,
    Parser,
    PiecewisePolynomial,
    PlanarSceneGraphVisualizer,
    Point,
    PointCloud,
    Rgba,
    RigidTransform,
    RotationMatrix,
    SceneGraph,
    Simulator,
    Solve,
    Sphere,
    StartMeshcat,
    TrajectorySource,
    Variable,
    eq,
)

from underactuated import ConfigureParser, running_as_notebook
from underactuated.jupyter import AdvanceToAndVisualize

In [ ]:
# Start the visualizer (run this cell only once, each instance consumes a port)
meshcat = StartMeshcat()

# Direct Transcription (using Mathematical Program directly)

In [ ]:
def double_integrator():
    # Discrete-time approximation of the double integrator.
    dt = 0.01
    A = np.eye(2) + dt * np.mat("0 1; 0 0")
    B = dt * np.mat("0; 1")

    prog = MathematicalProgram()

    N = 284  # Note: I had to do a manual "line search" to find this.

    # Create decision variables
    u = np.empty((1, N - 1), dtype=Variable)
    x = np.empty((2, N), dtype=Variable)
    for n in range(N - 1):
        u[:, n] = prog.NewContinuousVariables(1, "u" + str(n))
        x[:, n] = prog.NewContinuousVariables(2, "x" + str(n))
    x[:, N - 1] = prog.NewContinuousVariables(2, "x" + str(N))

    # Add constraints
    x0 = [-2, 0]
    prog.AddBoundingBoxConstraint(x0, x0, x[:, 0])
    for n in range(N - 1):
        # Will eventually be prog.AddConstraint(x[:,n+1] == A@x[:,n] + B@u[:,n])
        # See drake issues 12841 and 8315
        prog.AddConstraint(eq(x[:, n + 1], A.dot(x[:, n]) + B.dot(u[:, n])))
        prog.AddBoundingBoxConstraint(-1, 1, u[:, n])
    xf = [0, 0]
    prog.AddBoundingBoxConstraint(xf, xf, x[:, N - 1])

    result = Solve(prog)

    x_sol = result.GetSolution(x)
    assert result.is_success(), "Optimization failed"

    plt.figure()
    plt.plot(x_sol[0, :], x_sol[1, :])
    plt.xlabel("q")
    plt.ylabel("qdot")


double_integrator()

## DirectTranscription (using the DirectTranscription class)

Because this pattern of making decision variables that are indexed over time, adding the dynamic constraints, defining the running cost and constraints, is so common, we have wrappers in drake on top of `MathematicalProgram` which handle these details for you.

The optimization below is identical to the example above, but using this helper class.

In [ ]:
def dirtran_example():
    # Discrete-time approximation of the double integrator.
    dt = 0.01
    A = np.eye(2) + dt * np.mat("0 1; 0 0")
    B = dt * np.mat("0; 1")
    C = np.eye(2)
    D = np.zeros((2, 1))
    sys = LinearSystem(A, B, C, D, dt)

    N = 284
    x0 = [-2, 0]
    xf = [0, 0]

    dirtran = DirectTranscription(sys, sys.CreateDefaultContext(), N)
    prog = dirtran.prog()
    prog.AddBoundingBoxConstraint(x0, x0, dirtran.initial_state())
    prog.AddBoundingBoxConstraint(xf, xf, dirtran.final_state())
    dirtran.AddConstraintToAllKnotPoints(dirtran.input()[0] <= 1)
    dirtran.AddConstraintToAllKnotPoints(dirtran.input()[0] >= -1)

    result = Solve(prog)
    x_sol = dirtran.ReconstructStateTrajectory(result)
    assert result.is_success(), "Optimization failed"

    plt.figure()
    x_values = x_sol.vector_values(x_sol.get_segment_times())

    plt.plot(x_values[0, :], x_values[1, :])
    plt.xlabel("q")
    plt.ylabel("qdot")


dirtran_example()

One thing that I'm very proud of (it was a lot of work!) is the fact that drake is often smart enough to introspect your system, costs, and constraints and understand whether you have formulated a convex problem or a non-convex one.  The optimization above calls a convex optimization solver.  But if you had passed in a nonlinear system instead, it would have switched to calling a solver that supports nonlinear programming.